# Importing libraries

In [1]:
# importing necessary libraries
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Useful functions

In [2]:
# function to load the data
def load_data(filepath):
    return pd.read_csv(filepath, sep='\t', quoting=csv.QUOTE_NONE,names = ['label', 'message'])

# set of english stopwords
stop_words = set(stopwords.words('english'))

# function for preprocessing messages
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Stopword removal
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Converting all text to lowercase
    tokens = [token.lower() for token in tokens]
    
    # Removing empty strings
    tokens = [token for token in tokens if token != '']
    
    return tokens

# function for encoding ham as 0 and spam as 1
def encode(text):
    if text == 'spam':
        return 1
    elif text == 'ham':
        return 0
    
# function to split into train and test data
def split_data(df, seed = 42, train_size = 0.7, val_size = 0.15, test_size = 0.15):
    train_df, val_test_df = train_test_split(df, test_size=1-train_size, random_state = seed)
    val_df, test_df = train_test_split(val_test_df, test_size=test_size/(1-train_size), random_state = seed)
    return train_df, val_df, test_df

# function to store as a csv file
def store_as_csv(df, name):
    return df.to_csv(name, index=False)

# function to print variable distribution
def print_variable_distribution(data, file_name):
    print("Number of 0s in " + file_name + " - " + str(len(data) - data.spam.sum()))
    print("Number of 1s in " + file_name + " - " + str(data.spam.sum()))

# Load the data

In [5]:
# load the sms data
messages = load_data('data/raw_data')

In [6]:
messages.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [7]:
store_as_csv(messages,name='./data/raw_data.csv')

# Initialize dvc

In [8]:
!dvc init --subdir

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


Adding google drive as remote storage

In [47]:
# !dvc remote add --default myremote gdrive://1OONKIsu54eysQwijR4EaFTWV96e81NLB
# !dvc remote modify myremote gdrive_acknowledge_abuse true
# !dvc push

In [9]:
!git add .dvc/config
# !git commit -m "Adding Gdrive as Remote"

Store raw_data.csv using dvc

In [10]:
!git rm -r --cached 'data/raw_data.csv'
!dvc add data/raw_data.csv 
!git commit -m "Added raw_data.csv"

fatal: pathspec 'data/raw_data.csv' did not match any files


⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/utpalraj/coursework/AML/AppliedMachineLearning/Ass
                                                                                
!
  0%|          |Adding data/raw_data.csv to cache     0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /home/utpalraj/coursework0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  8.40file/s]

To track the changes with git, run:

	git add data/raw_data.csv.dvc data/.gitignore

To enable auto staging, run:

	dvc config core.autostage true
[detached HEAD c06e4a6] Added raw_data.csv
 3 files changed, 6 insertions(+)
 create mode 100644 Assignment2/.dvc/.gitignore
 create mo

In [11]:
!dvc status

Data and pipelines are up to date.                                              


# Data Preprocessing

In [12]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/utpalraj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/utpalraj/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/utpalraj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/utpalraj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Preprocessing message

In [13]:
messages['message'] = messages['message'].apply(preprocess_text)

In [14]:
messages.head(10)

,label,message
0,ham,"[go, jurong, point, ,, crazy, .., available, b..."
1,ham,"[ok, lar, ..., joking, wif, u, oni, ...]"
2,spam,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,"[u, dun, say, early, hor, ..., u, c, already, ..."
4,ham,"[nah, i, n't, think, go, usf, ,, life, around,..."
5,spam,"[freemsg, hey, darling, 's, 3, week, 's, word,..."
6,ham,"[even, brother, like, speak, ., they, treat, l..."
7,ham,"[as, per, request, 'melle, melle, (, oru, minn..."
8,spam,"[winner, !, !, as, valued, network, customer, ..."
9,spam,"[had, mobile, 11, month, ?, u, r, entitled, up..."


## Encode ham as 0 and spam as 1

In [15]:
messages['label'] = messages['label'].apply(encode)

# Splitting into Train, Validation and Test data

In [16]:
SEED1 = 2032
SEED2 = 2001

## using `SEED1`

In [17]:
# Split the data into training, validation, and testing sets
train_df, val_df, test_df = split_data(messages, seed=SEED1)

### Saving Train, Validation and Test Data as csv files

In [18]:
# Save the training, validation, and testing sets to CSV files
store_as_csv(train_df, name = './data/train.csv')
store_as_csv(val_df, name = './data/validation.csv')
store_as_csv(test_df, name = './data/test.csv')

In [19]:
!dvc status

Data and pipelines are up to date.                                              


In [ ]:
!dvc add "./data/train.csv"
!dvc add "./data/validation.csv"
!dvc add "./data/test.csv"
!git add .
!git commit -m "Added train, validation and test data for SEED1"

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/utpalraj/coursework/AML/AppliedMachineLearning/Ass
                                                                                
!
  0%|          |Adding data/train.csv to cache        0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /home/utpalraj/coursework0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 11.20file/s]

To track the changes with git, run:

	git add data/.gitignore data/train.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true
⠋ Checking graph                                       core>
Adding...                                                                    

## Using `SEED2`

In [28]:
# Split the data into training, validation, and testing sets
train_df, val_df, test_df = split_data(messages, seed=SEED2)

### Saving Train, Validation and Test Data as csv files

In [29]:
# Save the training, validation, and testing sets to CSV files
store_as_csv(train_df, name = './data/train.csv')
store_as_csv(val_df, name = './data/validation.csv')
store_as_csv(test_df, name = './data/test.csv')

In [30]:
!dvc status

data/train.csv.dvc:                                                             
	changed outs:
		modified:           data/train.csv
data/validation.csv.dvc:
	changed outs:
		modified:           data/validation.csv
data/test.csv.dvc:
	changed outs:
		modified:           data/test.csv


In [31]:
!dvc add "./data/train.csv"
!dvc add "./data/validation.csv"
!dvc add "./data/test.csv"
!git add .
!git commit -m "Added train, validation and test data for SEED2"

                                                                core>⠋ Checking graph
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/utpalraj/coursework/AML/AppliedMachineLearning/Ass
                                                                                
!
  0%|          |Adding data/train.csv to cache        0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /home/utpalraj/coursework0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 28.23file/s]

To track the changes with git, run:

	git add data/train.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true
⠋ Checking graph                                       core>
Adding...                                                           

# Target Variable Distribution of the First Version of Data

In [32]:
!git log --oneline

007735c (HEAD) Added train, validation and test data for SEED2
bb117d1 Added train, validation and test data for SEED1
3373ed2 Added raw_data.csv
6769486 Added raw_data.csv
c636675 Added raw_data.csv
2a43917 Added raw_data.csv
0d7987c Added raw_data.csv
5ae244a Added raw_data.csv
b6d5f0c Added train, validation and test data for SEED2
24ba264 Added train, validation and test data for SEED1
a89499a Added train, validation and test data for SEED1
412d939 Added raw_data.csv
ed4422c Added raw_data.csv
74cfbb1 Added raw_data.csv
07450eb Added raw_data.csv
bd49797 Added raw_data.csv
271529a Added raw_data.csv
af419cc Added raw_data.csv
57052c5 Added raw_data.csv
993a810 Adding Gdrive as Remote
3a04280 Added raw_data.csv
4f460cb Added raw_data.csv
d860e38 Added raw_data.csv
7d44a4e Added raw_data.csv
4b1db74 Added raw_data.csv
856daa6 Adding Gdrive as Remote
4e5e5e6 (origin/main, origin/HEAD) Minor correction
4277718 Corrected some minor errors
a60f4a4 Minor Change to train.ipynb
57f6a3c upda